In [4]:
import requests
import pandas as pd

# CityBikes

In [5]:
bike_df = pd.read_json('bike_df')

bike_df

,Station ID,Name,Latitude,Longitude,Number of Bikes
0,fb337bbed72e2be090071e199899b2be,Queen St E / Woodward Ave,43.665269,-79.319796,0
1,4ff88d5880e71aa40d34cfe5d09b0ca7,Primrose Ave / Davenport Rd,43.671420,-79.445947,3
2,a09c67c0b419654d907c9134b108e328,Queen St. E / Rhodes Ave.,43.666224,-79.317693,1
3,d6a9daee68070a8b106cfb598d81308c,Bond St / Queen St E,43.653236,-79.376716,12
4,8f8af40d9388c8a3962559e8681d3db7,Church St / Alexander St,43.663722,-79.380288,7
...,...,...,...,...,...
96,a7ca61373064c6581461132f1ae050de,Broadview Ave / Westwood Ave,43.684865,-79.356602,22
97,62a1c39596c3d512dfa04f1d29b81035,Fulton Ave / Pape Ave,43.683950,-79.347046,3
98,b960be1eba97f5aefc6ad39ecc93cb11,Cosburn Ave / Donlands Ave,43.690537,-79.341307,5
99,908b17c0d06011373777bace0a3cd74c,Beltline Trail / Chaplin Cres.,43.704461,-79.423122,8


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [18]:
API_key = 'fsq3dQYQp2ka71CYVbI6ehUtmigysRY/oygrq7j7KFRHyI8='
url = 'https://api.foursquare.com/v3/places/nearby'
radius = 1000  # Radius in meters
limit = 10

In [19]:
# Create a list to store the restaurant details
fq_rest_details = []

In [20]:
# Iterate over each row in the dataframe
for index, row in bike_df.iterrows():
    station_id = row['Station ID']
    latitude = row['Latitude']
    longitude = row['Longitude']

    params = {
        'll': f'{latitude},{longitude}',
        'radius': radius,
        'query': 'restaurant',
        'limit': limit
    }
    headers = {"Authorization": API_key}

    response = requests.get(url, params=params, headers=headers)

    fq_rest_data = response.json()

    # Extract the list of restaurants
    restaurants = fq_rest_data['results']

    # Iterate over each restaurant and extract the details
    for restaurant in restaurants:
        name = restaurant['name']
        location = restaurant['location']['formatted_address']
        latitude = restaurant['geocodes']['main']['latitude']
        longitude = restaurant['geocodes']['main']['longitude']

        # Append the restaurant details as a dictionary to the list
        fq_rest_details.append({
            'Station ID': station_id,
            'Retaurant name': name,
            'Location': location,
            'Latitude': latitude,
            'Longitude': longitude
        })

Put your parsed results into a DataFrame

In [21]:
# Create Foursquare restaurants DataFrame
fq_df = pd.DataFrame(fq_rest_details)

fq_df

,Station ID,Retaurant name,Location,Latitude,Longitude
0,fb337bbed72e2be090071e199899b2be,Gio Rana's Really Really Nice Restaurant,"1220 Queen St E, Toronto ON M4M 1L7",43.663345,-79.330418
1,fb337bbed72e2be090071e199899b2be,Occasions Restaurant,"30 Eastwood Rd (Gainsborough Road), Toronto ON...",43.673316,-79.318244
2,fb337bbed72e2be090071e199899b2be,Regency Restaurant Inc,"1423 Gerrard St E (Hiawatha St), Toronto ON M4...",43.672004,-79.322433
3,fb337bbed72e2be090071e199899b2be,Restaurant Tiflisi,"1970 Queen St E, Toronto ON M4L 1H8",43.669848,-79.301657
4,fb337bbed72e2be090071e199899b2be,Famous Indian Cuisine,"1437 Gerrard St E (at Ashdale Ave), Toronto ON...",43.672148,-79.321988
...,...,...,...,...,...
858,d2ce6ae839b7df2c912ab140253a5574,Restaurant Tiflisi,"1970 Queen St E, Toronto ON M4L 1H8",43.669848,-79.301657
859,d2ce6ae839b7df2c912ab140253a5574,Regency Restaurant Inc,"1423 Gerrard St E (Hiawatha St), Toronto ON M4...",43.672004,-79.322433
860,d2ce6ae839b7df2c912ab140253a5574,Ruyi Chinese Restaurant,"1029 Gerrard St E, Toronto ON M4M 1Z6",43.668711,-79.336972
861,d2ce6ae839b7df2c912ab140253a5574,Restaurant Asal,"1328 Danforth Ave, Toronto ON M4J 1M9",43.682393,-79.329406


In [22]:
fq_df.to_json('fq_df')

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [23]:
API_key = 'qekbhZMmgf57ynmxgZU-d7KA7OSmD-7BjDKsYH3hpcq-QBW2DgnDaQ-tA788OeYrgu_TLbmQKzx6NbyBStHJ-CvOyo3Go1Ok9aM458sYJ_AtBFdb6f1K9h9f1xB8ZHYx'
url = 'https://api.yelp.com/v3/businesses/search'
radius = 1000  # Radius in meters
limit = 10

In [24]:
# Create a list to store restaurant details
y_rest_details = []

In [25]:
# Iterate over each row in the dataframe
for index, row in bike_df.iterrows():
    station_id = row['Station ID']
    latitude = row['Latitude']
    longitude = row['Longitude']

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'radius': radius,
        'categories': 'restaurants',
        'limit': limit
    }
    headers = {
        'Authorization': f'Bearer {API_key}'
    }

    # Send API request to Yelp
    response = requests.get(url, params=params, headers=headers)
    y_rest_data = response.json()

    # Extract the list of businesses
    businesses = y_rest_data['businesses']

    # Iterate over each business and extract the details
    for business in businesses:
        name = business['name']
        location = ', '.join(business['location']['display_address'])
        latitude = business['coordinates']['latitude']
        longitude = business['coordinates']['longitude']
        rating = business['rating']

        # Create a dictionary with the restaurant details
        restaurant = {
            'Station ID': station_id,
            'Restaurant name': name,
            'Location': location,
            'Latitude': latitude,
            'Longitude': longitude,
            'Rating': rating
        }

        # Append the restaurant details to the list
        y_rest_details.append(restaurant)


Put your parsed results into a DataFrame

In [26]:
# Create Yelp restaurants DataFrame
yelp_df = pd.DataFrame(y_rest_details)

yelp_df

,Station ID,Restaurant name,Location,Latitude,Longitude,Rating
0,fb337bbed72e2be090071e199899b2be,Lake Inez,"1471 Gerrard Street E, Toronto, ON M4L 2A1, Ca...",43.672340,-79.320640,4.5
1,fb337bbed72e2be090071e199899b2be,O Sushi,"6 Coxwell Avenue, Toronto, ON M4L 3A7, Canada",43.666540,-79.316930,3.5
2,fb337bbed72e2be090071e199899b2be,Casa Di Giorgios,"1646 Queen St E, Toronto, ON M4L 1G3, Canada",43.666850,-79.315180,4.0
3,fb337bbed72e2be090071e199899b2be,The Burger's Priest,"1636 Queen Street E, Toronto, ON M4L 1G3, Canada",43.666700,-79.315585,3.5
4,fb337bbed72e2be090071e199899b2be,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",43.666270,-79.318020,4.5
...,...,...,...,...,...,...
996,d2ce6ae839b7df2c912ab140253a5574,Eastside Social,"1008 Queen Street E, Toronto, ON M4M 1K1, Canada",43.661432,-79.339113,4.5
997,d2ce6ae839b7df2c912ab140253a5574,Nodo Leslieville,"1192 Queen Street East, Toronto, ON M4M 1L6, C...",43.663110,-79.331620,4.0
998,d2ce6ae839b7df2c912ab140253a5574,Completo,"5 Coady Avenue, Toronto, ON M4M 2Y9, Canada",43.662633,-79.334058,4.5
999,d2ce6ae839b7df2c912ab140253a5574,Maple Leaf Tavern,"955 Gerrard Street E, Toronto, ON M4M 1Z4, Canada",43.668140,-79.339760,4.0


In [28]:
yelp_df.to_json('yelp_df')

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Base on the results in Part 2, Yelp API was able to generate more number of restaurants (more volume) comparing to Foursquare API using the same set of bike stations' data. 

Yelp also provides more complete data compared to Foursquare. Below are a few reasons why Yelp is considered to have more comprehensive data:

- Ratings and Reviews: Yelp includes user-generated ratings and reviews for businesses, allowing users to get insights into the quality and experience of the restaurants. Foursquare, on the other hand, does not provide this information in its basic search API.

- Additional Business Details: Yelp provides additional information such as phone numbers, website URLs, price range, and photos for businesses. These details can be useful for users who want to gather more information about a restaurant before making a decision. Foursquare does not offer this level of detail in its basic search API.

- Rich Content: Yelp allows businesses to add photos to their profiles, giving users a visual representation of the establishment. Foursquare does have photos, but they are generally limited to a single default image and may not provide as much visual context.

Overall, Yelp is known for its focus on user-generated content, including ratings, reviews, and detailed business information. This makes it a valuable resource for users who want more comprehensive insights into restaurants and other establishments. 